### IMPORTS

In [1]:
# Imports that will be using in this notebook

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Input
from tensorflow.keras import layers
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import pickle

### READING IN DATA

In [2]:
# Data Source
# https://www.kaggle.com/kingburrito666/cannabis-strains

In [3]:
# Reading in cannabis data

weed = pd.read_csv('cannabis.csv')

In [4]:
# Checking the whole dataframe and making sure I read it in correctly

weed

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


### DATA PREPROCESSING

In [5]:
# I know that there are some NaN values in the Flavor column so checking to see how many

weed[weed['Flavor'].isnull()]

,Strain,Type,Rating,Effects,Flavor,Description
103,Amethyst,hybrid,3.9,"Energetic,Creative,Giggly,Hungry,Happy",NaN,Amethyst is a hybrid cannabis strain from the ...
123,Arabian-Gold,sativa,4.1,"Euphoric,Tingly,Giggly,Sleepy,Creative",NaN,Arabian Gold is a heavy sativa strain of myste...
160,Bad-Azz-Kush,hybrid,5.0,None,NaN,Bad Azz Kush by Barney’s Farm was created with...
215,Birds-Eye,sativa,5.0,"Happy,Uplifted,Energetic,Euphoric,Relaxed",NaN,Birds Eye is a variety of Jack Herer grown by ...
364,Boombaye,hybrid,0.0,None,NaN,Boombaye by Mr. Mack’s Snack is a CBD-rich str...
379,Broke-Diesel,hybrid,5.0,"Uplifted,Euphoric,Creative,Energetic,Happy",NaN,Broke Diesel is a 50/50 hybrid marijuana strai...
484,Chem-Jong-Ill,hybrid,5.0,"Uplifted,Creative,Happy,Talkative,Energetic",NaN,This sativa-dominant cross is for cannabis pro...
574,Confidential-Wreck,hybrid,4.2,"Hungry,Relaxed,Tingly,Uplifted,Creative",NaN,"Confidential Wreck, also known as Lohan, is th..."
673,Do-Over-Og,hybrid,4.7,"Talkative,Uplifted,Creative,Euphoric,Happy",NaN,Do-Over OG is an indica-dominant strain that c...
679,Domino,indica,4.0,"Sleepy,Happy,Tingly,Uplifted,Euphoric",NaN,NaN


In [6]:
# Replacing the NaN values for none in the Flavor column

weed['Flavor'] = weed['Flavor'].replace(np.nan, 'none')

In [7]:
# Making sure that there are no NaN values left

weed[weed['Flavor'].isnull()]

,Strain,Type,Rating,Effects,Flavor,Description


In [8]:
# Combining Effects and Flavor into once column to use for one hot encoding

weed['Effect_Flavor'] = weed['Effects'] + ',' +  weed['Flavor'] # If you don't add the comma it just smooshes the words together

In [9]:
# Checking to making sure it added correctly

weed

,Strain,Type,Rating,Effects,Flavor,Description,Effect_Flavor
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"Creative,Energetic,Tingly,Euphoric,Relaxed,Ear..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"Relaxed,Aroused,Creative,Happy,Energetic,Flowe..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"Uplifted,Happy,Relaxed,Energetic,Creative,Spic..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Tingly,Creative,Hungry,Relaxed,Uplifted,Aprico..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","Happy,Relaxed,Euphoric,Uplifted,Talkative,Citr..."
...,...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...,"Happy,Uplifted,Relaxed,Euphoric,Energetic,Eart..."
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...,"Relaxed,Happy,Euphoric,Uplifted,Sleepy,Sweet,B..."
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...,"Relaxed,Sleepy,Talkative,Euphoric,Happy,Earthy..."
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...,"Relaxed,Sleepy,Euphoric,Happy,Hungry,Sweet,Ear..."


In [10]:
# Checking one row to better see all the items and making sure the comma seperated the words correctly

weed['Effect_Flavor'][0]

'Creative,Energetic,Tingly,Euphoric,Relaxed,Earthy,Sweet,Citrus'

In [11]:
# Looking at my new column

weed['Effect_Flavor'].describe()

count          2351
unique         2257
top       None,None
freq             73
Name: Effect_Flavor, dtype: object

### MODEL PREPERATION

In [12]:
# Split into X and y

X = weed['Effect_Flavor']
y = weed['Strain']

In [13]:
# Checking X shape

X.shape

(2351,)

In [14]:
# Checking y shape

y.shape

(2351,)

In [15]:
# Since the neural network cannot take in straight text I am going to vectorize my X data

# create the transformer
vect = CountVectorizer()

# build vocab
vect.fit(X)

# transform text
dtm = vect.transform(X)

In [16]:
# Checking to see that the feature names are correct

print(vect.get_feature_names())

['ammonia', 'apple', 'apricot', 'aroused', 'berry', 'blue', 'blueberry', 'butter', 'cheese', 'chemical', 'chestnut', 'citrus', 'coffee', 'creative', 'diesel', 'dry', 'earthy', 'energetic', 'euphoric', 'flowery', 'focused', 'fruit', 'giggly', 'grape', 'grapefruit', 'happy', 'herbal', 'honey', 'hungry', 'lavender', 'lemon', 'lime', 'mango', 'menthol', 'mint', 'minty', 'mouth', 'none', 'nutty', 'orange', 'peach', 'pear', 'pepper', 'pine', 'pineapple', 'plum', 'pungent', 'relaxed', 'rose', 'sage', 'skunk', 'sleepy', 'spicy', 'strawberry', 'sweet', 'talkative', 'tar', 'tea', 'tingly', 'tobacco', 'tree', 'tropical', 'uplifted', 'vanilla', 'violet', 'woody']


In [17]:
# Now I am going to use todense() to turn X into a matrix

X = dtm.todense()

In [18]:
# Make sure it turned into a matrix correctly

X

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [19]:
# I am using label encoder for y
# Label encoder turns y into a numeric representation of the data

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [20]:
# Check to make sure that y is an array now

y

array([   0,   18,    1, ..., 2347, 2348, 2349])

In [21]:
# Checking to make sure the X shape looks good

X.shape

(2351, 66)

In [22]:
# Checking to make sure the y shape looks good

y.shape

(2351,)

In [23]:
# Checking to make sure my X and y look good

print(X[1])
print(y[0:5])

[[0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
[ 0 18  1  2  3]


### CREATE NEURAL NETWORK

In [24]:
# Creating my own optimizer
# AdamamsGrad is using a super low learning_rate which means the changes between epochs will be smaller

AdamamsGrad = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    amsgrad=False,
)

In [25]:
# Instantiating my model type and creating the model architecture

model = Sequential()

model.add(Input(66))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2350, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=AdamamsGrad, 
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4288      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 2350)              603950    
Total params: 649,582
Trainable params: 649,582
Non-trainable params: 0
_________________________________________________________________


### TEST PREDICTION MODEL

In [26]:
# Fitting my model and getting my accuracy

model.fit(X,y, epochs=100)

Epoch 1/100
74/74 [==============================] - 0s 4ms/step - loss: 7.7792 - accuracy: 0.0000e+00
Epoch 2/100
74/74 [==============================] - 0s 4ms/step - loss: 7.7473 - accuracy: 0.0017
Epoch 3/100
74/74 [==============================] - 0s 4ms/step - loss: 7.7122 - accuracy: 0.0094
Epoch 4/100
74/74 [==============================] - 0s 4ms/step - loss: 7.6385 - accuracy: 0.0247
Epoch 5/100
74/74 [==============================] - 0s 4ms/step - loss: 7.5002 - accuracy: 0.0345
Epoch 6/100
74/74 [==============================] - 0s 4ms/step - loss: 7.2745 - accuracy: 0.0532
Epoch 7/100
74/74 [==============================] - 0s 4ms/step - loss: 6.9522 - accuracy: 0.0719
Epoch 8/100
74/74 [==============================] - 0s 4ms/step - loss: 6.5488 - accuracy: 0.1034
Epoch 9/100
74/74 [==============================] - 0s 4ms/step - loss: 6.0947 - accuracy: 0.1612
Epoch 10/100
74/74 [==============================] - 0s 4ms/step - loss: 5.6114 - accuracy: 0.2395
Epoch

### PICKLE THE MODEL